In [ ]:
pip install langgraph langchain-community langchain-anthropic tavily-python typing-extensions langchain-openai gradio


In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image, display, Markdown
from typing import List, Dict, Optional
from typing_extensions import TypedDict
from datetime import datetime
import json
import os
import re 
import threading
import gradio as gr
from dotenv import load_dotenv

load_dotenv()

class BlogState(TypedDict):
    suggested_topics: List[Dict]
    draft: List[str]
    draft_titles: List[str]
    selected_topic: Optional[str]
    last_research: List[Dict]
    schedule_post: List[Dict]

llm = ChatAnthropic(model="claude-3-haiku-20240307", max_tokens=4096)
tavily_tool = TavilySearchResults(max_results=5)


STATE_FILE_PATH = "blog_agent_memory.json"
def save_state_to_disk(state: BlogState, path: str = STATE_FILE_PATH):
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(state, f, indent=2, ensure_ascii=False)
    except Exception as e:
        print(f"Errore nel salvataggio dello stato: {e}")

def load_state_from_disk(path: str = STATE_FILE_PATH) -> BlogState:
    if os.path.exists(path):
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            return {
                "suggested_topics": data.get("suggested_topics", []),
                "draft": data.get("draft", []),
                "draft_titles": data.get("draft_titles", []),
                "selected_topic": data.get("selected_topic", []),
                "last_research": data.get("last_research", []),
                "schedule_post": data.get("schedule_post", [])
            }
        except Exception as e:
            print(f"Errore nel caricamento dello stato: {e}")
    return {
        "suggested_topics": [],
        "draft": [],
        "draft_titles": [],
        "selected_topic": [],
        "last_research": [],
        "schedule_post": []
    }

#|-------------------------------------------------------------------------------------------|

def suggest_topics(state: BlogState, user_input: str):
    recent_topics = state.get("draft_titles", [])[-10:]

    prompt = f"""**Task**: Genera **da 3 a 5 idee** per un post blog di tipo **'how-to'** (guida pratica), partendo dal tema: '{user_input}'. 
        Il post deve spiegare **come eseguire un'attività specifica**, dettagliando il processo e includendo suggerimenti utili.
        Il post deve essere chiaro, utile e adatto a un pubblico che sta cercando di **imparare o migliorare una competenza**.

        **Requisiti per l'idea**:
        - Deve includere procedure passo-passo, tecniche, o soluzioni pratiche a problemi comuni.
        - Deve includere argomenti che sono facilmente comprensibili e che possono essere implementati dai lettori con le risorse disponibili.
        - Può suggerire strumenti, materiali o risorse che potrebbero essere utili durante il processo descritto.

        **Vincoli**:
        1. Evitare questi topic recenti: {recent_topics}. Non ripetere argomenti trattati recentemente.
        2. Per ogni suggerimento fornire:
        - un **Titolo accattivante** che stimoli l'interesse del lettore e descriva chiaramente il contenuto della guida.
        - il **Tipo** del post (sempre "how-to" in questo caso).
        - una **Motivazione** (anche più di una frase) che spiega perché il post è utile e come risolverà un problema o semplificherà una attività per il lettore.
        3. Gli argomenti devono essere pratici e applicabili. **Esempi di categorie potrebbero includere**:
        - Tecniche manuali (es. "Come fare la manutenzione di un attrezzo da giardino").
        - Software e strumenti (es. "Come ottimizzare le performance di un sito web").
        - Attività quotidiane o di vita pratica (es. "Come organizzare un armadio in modo efficiente").
        4. Il contenuto deve essere scritto in modo che il lettore possa facilmente seguire le istruzioni **senza conoscenze pregresse**, includendo eventualmente consigli o risorse utili.
        5. Ogni idea deve essere originale e non troppo simile ad altri post già suggeriti.

        **Lingua dell'output**: l'intero risultato deve essere scritto in **italiano**.

        **Formato dell' output**:
        ```json
        [
            {{
                "title": "Titolo del post",
                "type": "how-to",
                "justification": "Descrizione dettagliata dell'argomento e del processo, e perché è utile per i lettori."
            }},
            ...
        ]
        ```
    """

    response = llm.invoke(prompt)
    try:
        content = response.content
        if '```json' in content:
            content = content.split('```json')[1].split('```')[0]
        suggestions = json.loads(content)

        for topic in suggestions:
            if not all(key in topic for key in ['title', 'type', 'justification']):
                raise ValueError("Formato topic non valido")
            
        display(Markdown(f"🟢 È stato scelto il seguente argomento: {user_input}"))

        display(Markdown("🟢 Topic suggeriti:"))
        for i, t in enumerate(suggestions, 1):
            display(Markdown(f"{i}. {t['title']} ({t['type']}) - {t['justification']}"))

        while True:
            try:
                selected_index = int(input("Scegli il numero del topic da usare: ")) - 1
                if 0 <= selected_index < len(suggestions):
                    selected_topic = suggestions[selected_index]["title"]
                    state["selected_topic"] = selected_topic
                    display(Markdown(f"✅ Topic selezionato: {selected_topic}"))
                    break
                else:
                    display(Markdown("⚠️ Selezione non valida. Riprova."))
            except ValueError:
                display(Markdown("⚠️ Inserisci un numero valido."))

        state["suggested_topics"] = suggestions
        save_state_to_disk(state)
        return state

    except (json.JSONDecodeError, ValueError) as e:
        print(f"Errore nel parsing dei suggerimenti: {str(e)}")
        display(Markdown("🔴 Errore: problemi nella generazione dei suggerimenti"))
        return {
            "suggested_topics": [{
                "title": "Errore: suggerimenti non disponibili",
                "type": "error",
                "justification": "Prova a riformulare la richiesta"
            }],
        }

#|----------------------------------------------------------------------------------------------------------------------------|
def classify_topic_with_llm(topic):
    categories = [
        "viaggi", "tecnologia", "cucina", "programmazione", "fai_da_te", 
        "moda", "arte", "musica", "scrittura", "sport", "cinema_e_tv",
        "fotografia", "finanza_personale", "automobili_e_motori"
    ]
    prompt = f"""Dato l'argomento: "{topic}", scegli **solo** una categoria dalla seguente lista:
        {', '.join(f'"{cat}"' for cat in categories)}.
        Rispondi solo con una delle parole esattamente come scritte sopra, senza aggiungere altro testo.
        """
    response = llm.invoke(prompt)
    return response if response in categories else "generica"

def research_topic(state: BlogState):
    if not state.get("selected_topic"):
        display(Markdown("⚠️ Nessun topic selezionato."))
        state["last_research"] = []
        return []

    topic = state["selected_topic"]
    query = f"{topic}. Guida pratica/tutorial dettagliato con istruzioni passo-passo"
    display(Markdown(f"Ricerca in corso: {query}"))

    try:
        raw_results = tavily_tool.invoke({
            "query": query,
            "search_depth": "advanced",
            "max_results": 5,
            "include_answer": False
        })
    except Exception:
        display(Markdown("🔴 Errore: Connessione al servizio di ricerca fallita."))
        state["last_research"] = []
        return []

    if not isinstance(raw_results, list) or not raw_results:
        display(Markdown("⚠️ Nessun risultato trovato."))
        state["last_research"] = []
        return []
    
    category = classify_topic_with_llm(topic)

    trusted_domains_by_category = {
        "viaggi": ["alpitour.it", "lonelyplanetitalia.it", "turismo.it", "viaggi.corriere.it", "doveviaggi.it", "skyscanner.it", "momondo.it", "tripadvisor.it", "montagna.tv", "mountainblog.it", "planetmountain.com", "trekking.it"],
        "tecnologia": ["wired.it", "hdblog.it", "tomshw.it", "tech.everyeye.it", "smartworld.it", "hwupgrade.it", "techradar.com", "androidworld.it", "youtube.com"],
        "cucina": ["giallozafferano.it", "ricettedellanonna.net", "cucchiaio.it", "cookaround.com", "lacucinaitaliana.it","fattoincasadabenedetta.it", "buttalapasta.it", "youtube.com"],
        "programmazione": ["aranzulla.it", "html.it", "stackoverflow.com", "geeksforgeeks.org", "dev.to", "w3schools.com", "docs.python.org", "developer.mozilla.org", "youtube.com"],
        "fai_da_te": ["bricolage-faidate.it", "ideegreen.it", "nigiara.it", "bricoportale.it", "lavorincasa.it","faidatemania.pianetadonna.it", "casafacile.it", "youtube.com", "giardinaggio.it"],
        "moda": ["vogue.it", "gqitalia.it", "elle.com/it", "vanityfair.it", "moda.corriere.it", "cosmopolitan.com/it", "youtube.com"],
        "arte": ["domestika.org", "artribune.com", "arte.it", "finestresullarte.info", "exibart.com", "artspecialday.com", "youtube.com"],
        "musica": ["rockol.it", "rollingstone.it", "ondarock.it", "allmusic.com", "metalitalia.com", "billboard.it" "youtube.com"],
        "scrittura": ["narrativamente.com", "thebookishbox.it", "pennablu.it", "scrittorincorso.it", "meloleggo.it", "manualediscrittura.it", "librierecensioni.com", "bookblister.com"],
        "sport": ["gazzetta.it", "corrieredellosport.it", "sportmediaset.it", "tuttosport.com", "figc.it", "calcio.com", "motogp.com", "espn.com", "youtube.com"],
        "cinema_e_tv": ["cineblog.it", "film.it", "mymovies.it", "comingsoon.it", "sky.it/cinema", "netflix.com", "youtube.com"],
        "fotografia": ["fotografareinitalia.it", "milanofotografia.it", "canon.it", "nikon.it", "pixaBay.com", "unsplash.com", "flickr.com"],
        "finanza_personale": ["finanzautile.it", "money.it", "ilsole24ore.com", "lavoce.info", "investireoggi.it", "wallstreetitalia.com"],
        "automobili_e_motori": ["automoto.it", "quattroruote.it", "motori.it", "autobest.co", "caradvice.com", "autoblog.com", "topgear.com"],
        "generica": ["wikipedia.org", "youtube.com", "ilpost.it", "repubblica.it", "corriere.it", "fanpage.it", "ansa.it"]
    }

    trusted_domains = trusted_domains_by_category.get(category, trusted_domains_by_category["generica"])

    evaluated_results = []
    for result in raw_results:
        try:
            domain = result.get("url", "").split("//")[-1].split("/")[0].lower()
            content = result.get("content", "")
            date = result.get("published_date", "")

            is_trusted = any(dom in domain for dom in trusted_domains)
            is_informative = len(content) > 100
            is_recent = any(year in date for year in ["2022", "2023", "2024", "2025"])

            score = sum([is_trusted, is_informative, is_recent])
            if score == 3:
                level = "alta"
            elif score == 2:
                level = "media"
            elif score == 1:
                level = "bassa"
            else:
                level = None

            if level:
                evaluated_results.append({
                    "title": result.get("title", "N/A"),
                    "url": result.get("url", ""),
                    "domain": domain,
                    "level": level,
                    "content": content,
                    "excerpt": content[:200] + "..."
                })
        except Exception:
            continue

    evaluated_results.sort(key=lambda x: ["alta", "media", "bassa"].index(x["level"]))
    final_results = evaluated_results[:3]

    state["last_research"] = final_results

    if final_results:
        display(Markdown("✅ Fonti selezionate:"))
        for res in final_results:
            display(Markdown(f"- [{res['title']}]({res['url']})  \n`{res['domain']}` | Affidabilità: **{res['level']}**"))
    else:
        display(Markdown("⚠️ Nessun risultato valido"))

    return final_results

#|----------------------------------------------------------------------------------------------------------------------------|
def modify_text(draft):
    modified_text = {"value": draft}
    done = threading.Event()

    def modify(draft_modificato):
        modified_text["value"] = draft_modificato
        done.set()
        return gr.update(value=draft_modificato, interactive=False), gr.update(value="Salvato", interactive=False)

    with gr.Blocks() as demo:
        textbox = gr.Textbox(label="Modifica il contenuto della bozza", value=draft, lines=40)
        submit_btn = gr.Button("Salva bozza", variant="primary")

        submit_btn.click(
            fn=modify,
            inputs=[textbox],
            outputs=[textbox, submit_btn]
        )
    threading.Thread(target=demo.launch(quiet=True), kwargs={"share": False}).start()
    done.wait()
    return modified_text["value"]

def ask_input(prompt, valid_options):
    while True:
        user_input = input(prompt).strip().lower()
        if user_input in valid_options:
            return user_input
        else:
            display(Markdown(f"⚠️ Input non valido, scrivi uno tra: {', '.join(valid_options)}"))

def generate_post(state: BlogState):
    if "selected_topic" not in state or not state["selected_topic"]:
        display(Markdown("⚠️ Nessun topic selezionato. Scegli un topic dai suggerimenti prima di procedere."))
        return

    title = state["selected_topic"]
    display(Markdown(f"📝 Creazione bozza per il topic: {title}"))

    available_moods = ["professionale", "divertente", "motivazionale", "ironico"]
    display(Markdown("Scegli uno stile di scrittura:"))
    for idx, mood in enumerate(available_moods, 1):
        display(Markdown(f"{idx}. {mood}"))
        
    valid_choices = [str(i) for i in range(1, len(available_moods) + 1)]
    choice = ask_input(f"Inserisci il numero dello stile desiderato (1-{len(available_moods)}): ",valid_choices)
    selected_mood = available_moods[int(choice) - 1]
    display(Markdown(f"🟢 Hai scelto lo stile: **{selected_mood}**"))
    display(Markdown("--------------------------------------------------"))
    
    display(Markdown("🔎 Avvio ricerca per le fonti!"))
    research_topic(state)

    research = state.get("last_research", [])
    if not research:
        display(Markdown("⚠️ Nessuna fonte disponibile."))
        return

    resources_text = "\n".join([ f"- {res['title']} ({res['url']})" for res in research])

    if not resources_text.strip():
        display(Markdown("⚠️ Nessuna fonte utile disponibile per il post."))
        return

    draft_prompt = f"""**Obiettivo**: Scrivi una **bozza completa** in lingua italiana per un post di tipo **"how-to"** (guida pratica) sul seguente argomento: **Topic**: "{title}"
        ---
        **Risorse da utilizzare**: {resources_text}
        ---
        **Stile richiesto**: {selected_mood}
        ---
        ### Linee guida per la scrittura del post:

        1. **Tono e stile**:
        - Usa un linguaggio coerente con lo stile: {selected_mood}.
        - Il tono deve essere pratico, amichevole e chiaro, adatto anche a lettori senza conoscenze pregresse.
        - Coinvolgi il lettore fin dall’inizio con esempi concreti e applicazioni pratiche dove possibile.

        2. **Struttura del contenuto**:
        - **Titolo**: aggiungi il titolo del topic scelto, cioè: {title}.
        - **Introduzione**: spiega brevemente l’utilità della guida, a chi è rivolta e cosa imparerà il lettore.
        - **Materiali o strumenti necessari** (se rilevanti): elenca tutto ciò che serve per seguire i passaggi.
        - **Passaggi dettagliati**:
            - Presenta ogni passaggio in ordine logico.
            - Per ogni step: spiega *cosa fare*, *perché è importante* e *come farlo al meglio*, includendo eventuali suggerimenti o trucchi.
        - **Consigli pratici**: aggiungi suggerimenti per migliorare l’esperienza o ottenere risultati migliori.
        - **Errori comuni da evitare**: segnala eventuali difficoltà o scelte sbagliate frequenti.
        - **Risorse aggiuntive** (opzionali): link utili, strumenti online, riferimenti.
        - **Conclusione**: incoraggia il lettore a provare, condividere la propria esperienza, o fare domande.

        3. **Formato**:
        - Usa **elenchi numerati** per i passaggi.
        - Evidenzia in **grassetto** i concetti chiave o le parole importanti.
        - Mantieni paragrafi brevi e scorrevoli. Esegui una descrizione più dettagliata ove lo ritieni necessario.
        ---
        ### Output atteso:
        - **Non aggiungere ulteriori titoli** come: "Ecco una bozza del post", "Nuova bozza dal titolo", ecc. Limitati a seguire la struttra fornita.
        - Fornisci il contenuto del post come **testo semplice in lingua italiana**, ben formattato e pronto per essere rivisto o pubblicato. 
        - **Non usare Markdown, codice, o formati strutturati come JSON.**
        """

    response = llm.invoke(draft_prompt)
    draft = response.content

    display(Markdown("✍️ Ecco una bozza del post:"))
    display(Markdown(draft))

    user_input = ask_input("Vuoi migliorare la bozza appena creata? (si/no): ", ["si", "no"])
    if user_input == "si":
        user_input_choice = ask_input("Vuoi aggiungere un feedback o modificare la bozza? Scegli (f/m): ", ["f", "m"])
        if user_input_choice == "m":
            updated_draft = modify_text(draft)
            user_confirm = ask_input("Vuoi memorizzare la bozza modificata? (si/No): ", ["si", "no"])
            if user_confirm == "si":
                state["draft"].append(updated_draft)

                match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", updated_draft, flags=re.IGNORECASE | re.DOTALL)
                if match:
                    extract_title = match.group(1).strip()
                else:
                    extract_title = updated_draft.strip().split("\n", 1)[0].strip()
                state.setdefault("draft_titles", []).append(extract_title)
                    
                state["selected_topic"] = None
                save_state_to_disk(state)
                display(Markdown("✅ Bozza modificata salvata."))
            else:
                state["draft"].append(draft)
                state.setdefault("draft_titles", []).append(title)
                state["selected_topic"] = None
                save_state_to_disk(state)
                display(Markdown("✅ Bozza originale salvata."))
        else:
            feedback = input("inserisci il tuo feedback: ").strip().lower()
            if not feedback:
                state["draft"].append(draft)
                state.setdefault("draft_titles", []).append(title)
                save_state_to_disk(state)
                display(Markdown("✅ Feedback non fornito, Bozza originale salvata."))
            else:
                display(Markdown("--------------------------------------------------"))
                display(Markdown(f"🟢 Hai scelto di revisione la bozza sulla base di questo feedback: {feedback}. Fase di revisione avviata.."))
                revision_prompt = f"""**Obiettivo**: Revisiona e migliora il contenuto della bozza di tipo **"how-to"** (guida pratica) tenendo conto del feedback fornito. 
                    **Lo stile, la struttura e la formattazione devono rimanere identici alla bozza originale.** 
                    ---
                    ### **Bozza originale**: {draft}
                    ---
                    ### **Feedback ricevuto**:{feedback}
                    ---
                    ### **Istruzioni per la revisione**:
                    1. **Migliora il contenuto testuale** mantenendo la struttura originale:
                    - Approfondisci i punti segnalati nel {feedback}, fornendo spiegazioni più chiare o dettagliate e pratiche.
                    - Assicurati che i passaggi siano logici, ben spiegati e facilmente comprensibili anche per principianti.
                    - Integra esempi pratici, consigli utili o note aggiuntive dove opportuno.
                    - Mantieni la **coerenza con la bozza originale fornita sopra**: migliora senza stravolgere, a meno che il **feedback lo richieda espressamente**.

                    2. **Mantieni stile e tono**:
                    - Rispetta lo stile e il tono originale richiesto: **{selected_mood}**.

                    3. **Mantieni la struttura della bozza originale**:
                    - Integra le modifiche apportate al contenuto rispettando la seguente struttura:
                        - **'Titolo:'**
                        - **'Introduzione:'**
                        - **'Materiali o strumenti necessari:'**
                        - **'Passaggi dettagliati:'**
                        - **'Consigli pratici:'**
                        - **'Errori comuni da evitare:'**
                        - **'Risorse aggiuntive** (opzionali):'
                        - **'Conclusione:'**
                    - **Non aggiungere ulteriori titoli** come: "Revisione del post", "Nuova bozza revisionata", ecc. Limitati ad inserire il Titolo e a seguire la struttra fornita.

                    4. **Formato**:
                    - Se lo ritini opportuno usa **elenchi numerati** per i passaggi.
                    - Evidenzia in **grassetto** i concetti chiave o le parole importanti.

                    5. **Revisione linguistica e formattazione**:
                    - Correggi eventuali errori grammaticali, sintattici o stilistici.
                    ---
                    ### **Output atteso**:
                    - Fornisci il contenuto del post come **testo semplice in lingua italiana**, ben formattato e pronto per essere rivisto o pubblicato. 
                    - **Non usare Markdown, codice, o formati strutturati come JSON.**
                    """
                
                response = llm.invoke(revision_prompt)
                updated_draft = response.content
                display(Markdown("✍️ Bozza Revisionata:"))
                display(Markdown(updated_draft))
                
                user_input_confirm = ask_input("Vuoi memorizzare la bozza revisionata? (si/no): ", ["si", "no"])
                if user_input_confirm == "si":
                    state["draft"].append(updated_draft)

                    match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", updated_draft, flags=re.IGNORECASE | re.DOTALL)
                    if match:
                        extract_title = match.group(1).strip()
                    else:
                        extract_title = updated_draft.strip().split("\n", 1)[0].strip()
                    state.setdefault("draft_titles", []).append(extract_title)

                    state["selected_topic"] = None
                    save_state_to_disk(state)
                    display(Markdown("✅ Bozza revisionata salvata."))
                else:
                    state["draft"].append(draft)
                    state.setdefault("draft_titles", []).append(title)
                    state["selected_topic"] = None
                    save_state_to_disk(state)
                    display(Markdown("✅ Bozza originale salvata."))

    else:
        state["draft"].append(draft)
        display(Markdown("✅ Bozza originale salvata."))
        state.setdefault("draft_titles", []).append(title)
        state["selected_topic"] = None
        save_state_to_disk(state)
        display(Markdown("🔁 Seleziona ora un nuovo topic per generare un altro post."))

#|----------------------------------------------------------------------------------------------------------------------------|

def show_state(state: BlogState):
    display(Markdown("🟢 Stato attuale del BlogAgent"))
    drafts = state.get("draft", [])

    if drafts:
        display(Markdown("✍️ Bozze salvate:"))
        for idx, draft in enumerate(drafts, 1):
            display(Markdown(f"**Bozza {idx}**"))
            display(Markdown(draft))
            display(Markdown("-----------------------------------------------"))

        valid_options = [str(i) for i in range(1, len(drafts) + 1)] + ["no"]
        user_choice = ask_input( f"Inserisci il numero della bozza da eliminare (1 a {len(drafts)}) oppure 'no' per saltare: ", valid_options)
        if user_choice != "no":
            idx = int(user_choice)
            removed = drafts.pop(idx - 1)

            match = re.search(r"\*?\*?\s*Titolo\s*\*?\*?:\s*(.+?)(?:\n{2,}|$)", removed, flags=re.IGNORECASE | re.DOTALL)
            if match:
                removed_title = match.group(1).strip()
            else:
                removed_title = removed.strip().split("\n", 1)[0].strip()

            if removed_title in state["draft_titles"]:
                state["draft_titles"].remove(removed_title)

            filtered_schedule = []
            for entry in state.get("schedule_post", []):
                if not re.fullmatch(rf"{re.escape(removed_title)}", entry["title"].strip(), flags=re.IGNORECASE):
                    filtered_schedule.append(entry)

            state["schedule_post"] = filtered_schedule
            
            save_state_to_disk(state)

            display(Markdown(f"🗑️ Bozza {idx} eliminata."))
        else:
            display(Markdown("✅ Nessuna bozza eliminata."))
    else:
        display(Markdown("📝 Nessuna post generata. Seleziona prima il topic dai suggerimenti!"))

#|----------------------------------------------------------------------------------------------------------------------------|

def publication_calendar(state: BlogState):

    display(Markdown("📆 Calendario di pubblicazione dei post"))
    if not state.get("draft_titles"):
        display(Markdown("⚠️ Nessuna bozza salvata disponibile per pianificare il calendario."))
        return state

    today_str = datetime.now().strftime("%d/%m/%Y")
    today = datetime.strptime(today_str, "%d/%m/%Y").date()
    calendario = state.get("schedule_post", []).copy() 

    sched_title = {entry["title"] for entry in state["schedule_post"]}

    for titolo in state["draft_titles"]:
        if titolo in sched_title:
            continue

        prompt = f"""Agisci come se fossi un esperto di content marketing e content strategist.
            Il tuo compito è decidere la **migliore data di pubblicazione** per un post di blog con il seguente titolo: "{titolo}"

            ### **Criteri da considerare**:
            - **Contenuto** del post: valuta se è legato a particolari stagioni, eventi o periodi dell’anno.
            - **Stagionalità e rilevanza**: adatta la data in base al contesto stagionale (es. maglioni in autunno/inverno, attività all’aperto in primavera/estate).
            - **Distribuzione realistica**: evita di concentrare tutte le pubblicazioni in pochi giorni.

            ### **Requisiti**:
            - La data **non deve essere antecedente a oggi ({today_str})**.
            - Restituisci **solo una singola data futura**, in **formato `DD/MM/YYYY`** (giorno/mese/anno).
            - Non aggiungere spiegazioni o testo extra: solo la data.

            ### **Esempi di errori da evitare**:
            - Proporre una data nel passato.
            - Scegliere una data incoerente col tipo di contenuto (es. maglioni in piena estate).
            """
        
        response = llm.invoke(prompt)
        pubblication_date = response.content.strip()

        calendario.append({
            "date": pubblication_date,
            "title": titolo,
            "status": "schedulato"
        })

    state["schedule_post"] = calendario

    for entry in state["schedule_post"]:
        try:
            scheduled_date = datetime.strptime(entry["date"], "%d/%m/%Y").date()
            if scheduled_date <= today and entry["status"] == "schedulato":
                entry["status"] = "pubblicato"
        except Exception as e:
            print(f"Errore nella data per il post '{entry['title']}': {e}")

    for entry in state["schedule_post"]:
        status_icon = "✅" if entry["status"] == "pubblicato" else "🕒"
        display(Markdown(f"- `{entry['date']}` → **{entry['title']}** | stato: {entry['status']} {status_icon}"))

    save_state_to_disk(state)
    return state

In [ ]:
def blogManager():
    state = load_state_from_disk()

    while True:
        display(Markdown("|---------- Benvenuto nel BlogAgent ----------|"))
        display(Markdown("1. Genera suggerimenti\n2. Crea post\n3. Mostra stato\n4. Pianifica il calendario\n5. Esci"))
        display(Markdown("|---------------------------------------------|"))
        choice = input("Seleziona una opzione: ").strip()

        if choice == "5":
            break
        try:
            if choice == "1":
                user_input = input("\nInserisci l'argomento della guida pratica: ").strip()
                state = suggest_topics(state, user_input)
            elif choice == "2":
                generate_post(state)
            elif choice == "3":
                show_state(state)
            elif choice == "4":
                state = publication_calendar(state)
            else:
                display(Markdown("⚠️ Scelta non valida"))
        except Exception as e:
            display(Markdown(f"🔴 Errore: {str(e)}"))

if __name__ == "__main__":
    blogManager()

In [ ]:
workflow = StateGraph(BlogState)
#|---------------------------------------------------------Definisco i nodi---------------------------------------------------------|
workflow.add_node("Caricamento dati", load_state_from_disk)
workflow.add_node("Menù", blogManager)
workflow.add_node("Genera suggerimenti", suggest_topics)
workflow.add_node("Seleziona topic", lambda state: (state, []))
workflow.add_node("Creazione bozza", generate_post)
workflow.add_node("Scelta stile", lambda state: (state, []))
workflow.add_node("Ricerca fonti", generate_post)
workflow.add_node("Bozza", lambda state:(state, []))
workflow.add_node("Review", lambda state: (state, []))
workflow.add_node("Modifica post", modify_text)
workflow.add_node("Feedback post", lambda state: (state, []))
workflow.add_node("Stato dell'agente", show_state)
workflow.add_node("Elimina bozza", lambda state: (state, []))
workflow.add_node("Mostra calendario", publication_calendar)
workflow.add_node("Esci", lambda state: (state, []))

#|---------------------------------------------------------Grafo e partenza---------------------------------------------------------|
workflow.add_edge(START, "Caricamento dati")

workflow.add_edge("Caricamento dati", "Menù")
workflow.add_conditional_edges(
    "Menù",
    lambda state : "1",
    {
        "1": "Genera suggerimenti", 
        "2": "Creazione bozza",  
        "3": "Stato dell'agente",  
        "4": "Mostra calendario", 
        "5": "Esci",
        "errore": "Menù" 
    }
)

#|---------------------------------------------------------Scelta dei suggerimenti e selezione topic---------------------------------------------------------|
workflow.add_conditional_edges(
    "Genera suggerimenti",
    lambda state: "Seleziona topic" if state.get("suggested_topics") else "Menù",
    {
        "topic": "Seleziona topic", 
        "errore": "Menù"
    }
)
workflow.add_edge("Seleziona topic", "Menù")

#|---------------------------------------------------------Creazione post e modifica---------------------------------------------------------|
workflow.add_conditional_edges(
    "Creazione bozza",
    lambda state: "Scelta stile" if state.get("selected_topic") else "Menù",
    {
        "Scelta stile": "Scelta stile", 
        "no topic": "Menù"
    }
)
workflow.add_edge("Scelta stile", "Ricerca fonti")
workflow.add_edge("Ricerca fonti", "Bozza")
workflow.add_conditional_edges(
    "Bozza",
    lambda state: "1",
    {
        "no modifica": "Menù",
        "si modifica": "Review"
    }
)
workflow.add_conditional_edges(
    "Review",
    lambda state: "1",
    {
        "m": "Modifica post",
        "f": "Feedback post"
    }
)
workflow.add_edge("Modifica post", "Menù")
workflow.add_edge("Feedback post", "Menù")

#|---------------------------------------------------------Stato---------------------------------------------------------|
workflow.add_conditional_edges(
    "Stato dell'agente",
    lambda state: "1",
    {
        "no": "Menù",
        "num_bozza": "Elimina bozza"
    }
)
workflow.add_edge("Elimina bozza", "Menù")

#|---------------------------------------------------------Calendario---------------------------------------------------------|
workflow.add_edge("Mostra calendario", "Menù")

#|---------------------------------------------------------Esci---------------------------------------------------------|
workflow.add_edge("Esci", END)

blog_agent = workflow.compile()
display(Image(blog_agent.get_graph().draw_mermaid_png()))